In [19]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()
#display(melb_df.info())

# # Задание 1.1
# melb_df['Date'] = pd.to_datetime(melb_df['Date'])
# quarter_sold = melb_df['Date'].dt.quarter
# quarter_sold.value_counts()

# # Задание 1.2
# cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
# max_unique_count = 150 # задаём максимальное число уникальных категорий
# for col in melb_df.columns: # цикл по именам столбцов
#     if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
#         melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
# display(melb_df.info())

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [20]:
# 2. Сортировка данных в DataFrame

# СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА:
melb_df.sort_values(by='Price').head(10)
melb_df.sort_values(by='Date', ascending=False)

# СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ:
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

# КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

# Задание 2.2
#melb_df.sort_values(by='AreaRatio', ascending=False)
# print(melb_df.loc[1558])

# ПРАВИЛЬНЫЙ ОТВЕТ
int(melb_df.sort_values(
    by='AreaRatio', 
    ignore_index=True,
    ascending=False
).loc[1558, 'BuildingArea'])


# # Задание 2.3
# mask1 = melb_df['Type'] == 'townhouse'
# mask2 = melb_df['Rooms'] > 2
# int(melb_df[mask1&mask2].sort_values(
#     by=['Rooms', 'MeanRoomsSquare'],
#     ascending=[True, False],
#     ignore_index=True
# ).loc[18, 'Price'])

126

In [ ]:
# 3. Группировка данных в DataFrame:
# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ:

melb_df.groupby(by='Type').mean(numeric_only=True)
melb_df.groupby('Type')['Price'].mean()
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

melb_df.groupby('MonthSale')['Price'].agg('describe')

melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set])

# Задание 3.1
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

# Задание 3.2
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

# Задание 3.3
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= melb_df['Date']) & (melb_df['Date']<= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

In [22]:
# 4. Сводные таблицы:
# МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ:
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

melb_df.groupby(['Rooms', 'Type'])['Price'].mean()
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

# МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

# МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
) 

# ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns
display(pivot['mean']['unit'])

mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

print(list(filtered_pivot.index))

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

median                        mean              \
Type                   house townhouse  unit       house   townhouse   
Regionname                                                             
Southern Metropolitan  586.0     246.0   0.0  569.643881  278.858824   
Western Metropolitan   531.0     198.0  62.0  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

['Southern Metropolitan', 'Western Metropolitan']


In [8]:
# # Задание 4.2
# melb_df.pivot_table(
#     values='BuildingArea',
#     index='Rooms',
#     columns='Type',
#     aggfunc=['median'],
#     fill_value=0
# )

# # Задание 4.3
# melb_df.pivot_table(
#     values='Price',
#     index='SellerG',
#     columns='Type',
#     aggfunc=['median'],
#     fill_value=0
# )
# ПРАВИЛЬНЫЙ ОТВЕТ
pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
)
max_unit_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_unit_price].index[0])

Nick


In [4]:
# 5. Объединение DataFrame: знакомимся с новыми данными:
import pandas as pd

# Задание 5.1 - 5.3

movies = pd.read_csv('data/movies.csv')
movies.head()
display(movies.info())

ratings1 = pd.read_csv('data/ratings1.csv')
ratings1.nunique()

dates = pd.read_csv('data/dates.csv')
dates['date'] = pd.to_datetime(dates['date'])
dates['date'].dt.year.max


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


None

<bound method Series.max of 0         2000
1         2000
2         2000
3         2000
4         2000
          ... 
100831    2017
100832    2017
100833    2017
100834    2017
100835    2017
Name: date, Length: 100836, dtype: int32>

In [34]:
# 6. Объединение DataFrame: concat:

# ratings1 = pd.read_csv('data/ratings1.csv')
# ratings2 = pd.read_csv('data/ratings2.csv')

# ratings = pd.concat(
#     [ratings1, ratings2],
#     ignore_index=True
# )
# display(ratings)

# print('Число строк в таблице ratings: ', ratings.shape[0])
# print('Число строк в таблице dates: ', dates.shape[0])
# print(ratings.shape[0] == dates.shape[0])

# display(ratings1.tail(1))
# display(ratings2.head(1))

# ratings = ratings.drop_duplicates(ignore_index=True)
# print('Число строк в таблице ratings: ', ratings.shape[0])

# ratings_dates = pd.concat([ratings, dates], axis=1)
# display(ratings_dates.tail(7))

# ЗАДАНИЕ 6.3
import pandas as pd
import os
def concat_user_files(path):
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data], axis=0, ignore_index=True)
    data = data.drop_duplicates()
    return data


In [14]:
# 7. Объединение DataFrame: join, merge:

# joined_false = ratings_dates.join(
#     movies,
#     rsuffix='_right',
#     how='left'
# )
# display(joined_false)

# joined_false = ratings_dates.join(
#     movies,
#     rsuffix='_left',
#     how='right'
# )
# display(joined_false)

# joined = ratings_dates.join(
#     movies.set_index('movieId'),
#     on='movieId',
#     how='left'
# )
# display(joined.head())

merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


In [ ]:
# Задание 7.5 (External resource)
import pandas as pd

items_df = pd.DataFrame({
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394],
            'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
            'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
        })

purchase_df = pd.DataFrame({
            'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132],
            'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
        })

# Введите свое решение ниже
merged = items_df.merge(
    purchase_df,
    on='item_id',
    how='left'
)

# ПРАВИЛЬНЫЙ ОТВЕТ:
merged = items_df.merge(purchase_df, how='inner', on='item_id')
income = (merged['price'] * merged['stock_count']).sum()
display(merged)
print(income)


In [24]:
# 8. Закрепление знаний:

# Задание 8.1
#библиотека для регулярных выражений
import re
import pandas as pd
joined = pd.read_csv('data/ratings_movies.csv') 

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
 
joined['year_release'] = joined['title'].apply(get_year_release)
joined.info()
display(joined)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0
...,...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller,2017.0
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller,2017.0
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror,2017.0
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi,2017.0


In [30]:
# Задание 8.2
mask = joined['year_release'] == 1999
joined[mask].groupby('title')['rating'].mean().sort_values()
#display(joined)

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [31]:
# Задание 8.3
mask = joined['year_release'] == 2010
joined[mask].groupby('genres')['rating'].mean().sort_values()


genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [ ]:
# Задание 8.4
joined.groupby('userId')['genres'].nunique().sort_values(ascending = False)

In [56]:
# Задание 8.5
joined.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [58]:
# Задание 8.6
mask = joined['year_release'] == 2018
grouped = joined[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [ ]:
# ЗАДАНИЕ 8.7
import pandas as pd

joined = pd.read_csv('data/ratings_movies.csv') 
joined['date'] = pd.to_datetime(joined['date'])
joined['year_rating'] = joined['date'].dt.year

pivot = joined.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
display(pivot)

In [43]:
# ЗАДАНИЕ 8.8
import pandas as pd
orders_df = pd.read_csv('data/orders.csv')
product_df = pd.read_csv('data/products.csv')

orders_products = orders_df.merge(
    product_df, 
    left_on='ID товара',
    right_on ='Product_ID',
    how='left')
orders_products[orders_products['Name'].isna()]

# ЗАДАНИЕ 8.9
orders_products[orders_products['Отменен'] == 'Да']['Name']

# ЗАДАНИЕ 8.10
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество'] 
orders_products[orders_products['Оплачен'] == 'Да'].groupby('ID Покупателя')['Profit'].sum().sort_values(ascending=False)


KeyError: 'Product_ID'